In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
spark = SparkSession.builder.appName('Missing').getOrCreate()
training = spark.read.csv('file.csv', header=True , inferSchema=True)
training.show()
training.columns

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|dashrath| 32|        10| 30000|
| Sourabh| 24|         9| 25000|
|  Deepak| 33|         7| 15000|
|    Neha| 23|         6| 16000|
|  Rohini| 27|        11| 50000|
|  Sarika| 31|        12| 60000|
|  Ganesh| 32|        12| 70000|
+--------+---+----------+------+



['Name', 'Age', 'Experience', 'Salary']

In [ ]:
# Create a new column outputCol:"Independant Feature" with inputCol:" Age, Experience"
feature=VectorAssembler(inputCols=['Age','Experience'], outputCol="Independant feature")
output=feature.transform(training)
output.show()

+--------+---+----------+------+-------------------+
|    Name|Age|Experience|Salary|Independant feature|
+--------+---+----------+------+-------------------+
|dashrath| 32|        10| 30000|        [32.0,10.0]|
| Sourabh| 24|         9| 25000|         [24.0,9.0]|
|  Deepak| 33|         7| 15000|         [33.0,7.0]|
|    Neha| 23|         6| 16000|         [23.0,6.0]|
|  Rohini| 27|        11| 50000|        [27.0,11.0]|
|  Sarika| 31|        12| 60000|        [31.0,12.0]|
|  Ganesh| 32|        12| 70000|        [32.0,12.0]|
+--------+---+----------+------+-------------------+



In [ ]:
finaldata=output.select("Independant feature","Salary")
finaldata.show()

+-------------------+------+
|Independant feature|Salary|
+-------------------+------+
|        [32.0,10.0]| 30000|
|         [24.0,9.0]| 25000|
|         [33.0,7.0]| 15000|
|         [23.0,6.0]| 16000|
|        [27.0,11.0]| 50000|
|        [31.0,12.0]| 60000|
|        [32.0,12.0]| 70000|
+-------------------+------+



In [ ]:
#Train 75% and Test 25% data
train_data,test_data=finaldata.randomSplit([0.75,0.25])
reg=LinearRegression(featuresCol='Independant feature',labelCol='Salary')
#Fit the data
reg=reg.fit(train_data)
#Display the coefficients
reg.coefficients

DenseVector([17.2973, 7607.9279])

In [ ]:
#Predict and evaluate
Pred_result=reg.evaluate(test_data)
#Predict the results and show
Pred_result.predictions.show()
print("(MeanAbsoluteError: ",Pred_result.meanAbsoluteError, ", MeanSquaredError: ",Pred_result.meanSquaredError,")")

+-------------------+------+------------------+
|Independant feature|Salary|        prediction|
+-------------------+------+------------------+
|        [27.0,11.0]| 50000| 48006.84684684687|
|        [32.0,10.0]| 30000| 40485.40540540542|
|        [32.0,12.0]| 70000|55701.261261261294|
+-------------------+------+------------------+

(MeanAbsoluteError:  8925.765765765753 , MeanSquaredError:  106123438.50877883 )
